In [1]:
import plotly.express as px
import numpy as np

In [10]:
'''
cribbed from https://github.com/cadCAD-org/cadCAD-tools/blob/main/notebooks/mpp.ipynb
Annotation of a simple mpp model from 
'''

TIMESTEPS = 100
SAMPLES = 10

## Sets the initial conditions for the model 
initial_conditions = {
    'prey_population': 100,
    'predator_population': 15
}

### Parameters for the experiment/model
### arrays by default will generate a parameter sweep
params = {
    "prey_birth_rate": [1.0],
    "predator_birth_rate": [0.01],
    "predator_death_const": [1.0],
    "prey_death_const": [0.03],
    # Precision of the simulation. Lower is more accurate / slower
    "dt": [0.01, 0.1, 0.05]
}

### Policy methods 
def p_predator_births(params, step, sL, s):
    dt = params['dt']
    predator_population = s['predator_population']
    prey_population = s['prey_population']
    birth_fraction = params['predator_birth_rate'] + \
        np.random.random() * 0.0002
    births = birth_fraction * prey_population * predator_population * dt
    return {'add_to_predator_population': births}


def p_prey_births(params, step, sL, s):
    dt = params['dt']
    population = s['prey_population']
    birth_fraction = params['prey_birth_rate'] + np.random.random() * 0.1
    births = birth_fraction * population * dt
    return {'add_to_prey_population': births}


def p_predator_deaths(params, step, sL, s):
    dt = params['dt']
    population = s['predator_population']
    death_rate = params['predator_death_const'] + np.random.random() * 0.005
    deaths = death_rate * population * dt
    return {'add_to_predator_population': -1.0 * deaths}


def p_prey_deaths(params, step, sL, s):
    dt = params['dt']
    death_rate = params['prey_death_const'] + np.random.random() * 0.1
    prey_population = s['prey_population']
    predator_population = s['predator_population']
    deaths = death_rate * prey_population * predator_population * dt
    return {'add_to_prey_population': -1.0 * deaths}

### State update methods
def s_prey_population(params, step, sL, s, _input):
    y = 'prey_population'
    x = s['prey_population'] + _input['add_to_prey_population']
    return (y, x)


def s_predator_population(params, step, sL, s, _input):
    y = 'predator_population'
    x = s['predator_population'] + _input['add_to_predator_population']
    return (y, x)

### Structure the state update blocks
partial_state_update_blocks = [
    {
        'label': 'Something parameters',
        'policies': {
            'predator_births': p_predator_births,
            'prey_births': p_prey_births,
            'predator_deaths': p_predator_deaths,
            'prey_deaths': p_prey_deaths,
        },
        'variables': {
            'predator_population': s_prey_population,
            'prey_population': s_predator_population
        }
    },
    {
        'label': 'Do Nothing',
        'policies': {
            
        },
        'variables': {
            
        }
    }
]

In [11]:
from cadCAD_tools import easy_run

df = easy_run(initial_conditions,
              params,
              partial_state_update_blocks,
              TIMESTEPS,
              SAMPLES,
              assign_params=True,
              drop_substeps=False)
df


                  ___________    ____
  ________ __ ___/ / ____/   |  / __ \
 / ___/ __` / __  / /   / /| | / / / /
/ /__/ /_/ / /_/ / /___/ ___ |/ /_/ /
\___/\__,_/\__,_/\____/_/  |_/_____/
by cadCAD

cadCAD Version: 0.4.28
Execution Mode: local_proc
Simulation Dimensions:
Entire Simulation: (Models, Unique Timesteps, Params, Total Runs, Sub-States) = (1, 100, 5, 40, 2)
     Simulation 0: (Timesteps, Params, Runs, Sub-States) = (100, 5, 40, 2)
Execution Method: local_simulations
Execution Mode: parallelized
Total execution time: 5.42s


,prey_population,predator_population,simulation,subset,run,substep,timestep,prey_birth_rate,predator_birth_rate,predator_death_const,prey_death_const,dt
0,100.000000,15.000000,0,0,1,0,0,1.0,0.01,1.0,0.03,0.01
1,99.966359,15.000303,0,0,1,1,1,1.0,0.01,1.0,0.03,0.01
2,99.966359,15.000303,0,0,1,2,1,1.0,0.01,1.0,0.03,0.01
3,99.970543,15.000089,0,0,1,1,2,1.0,0.01,1.0,0.03,0.01
4,99.970543,15.000089,0,0,1,2,2,1.0,0.01,1.0,0.03,0.01
...,...,...,...,...,...,...,...,...,...,...,...,...
8035,NaN,NaN,0,3,10,2,98,1.0,0.01,1.0,0.03,0.50
8036,NaN,NaN,0,3,10,1,99,1.0,0.01,1.0,0.03,0.50
8037,NaN,NaN,0,3,10,2,99,1.0,0.01,1.0,0.03,0.50
8038,NaN,NaN,0,3,10,1,100,1.0,0.01,1.0,0.03,0.50


In [9]:

### we can query to extract specific instaces of a parameter set
df.query('dt == 0.01')

fig = px.line(df.query('dt == 0.1'),
              x=df.prey_population,
              y=df.predator_population,
              color=df.run.astype(str))

fig.show()

,prey_population,predator_population,simulation,subset,run,substep,timestep,prey_birth_rate,predator_birth_rate,predator_death_const,prey_death_const,dt
0,100.000000,15.000000,0,0,1,0,0,1.0,0.01,1.0,0.03,0.01
1,99.966359,15.000303,0,0,1,1,1,1.0,0.01,1.0,0.03,0.01
2,99.966359,15.000303,0,0,1,2,1,1.0,0.01,1.0,0.03,0.01
3,99.970543,15.000089,0,0,1,1,2,1.0,0.01,1.0,0.03,0.01
4,99.970543,15.000089,0,0,1,2,2,1.0,0.01,1.0,0.03,0.01
...,...,...,...,...,...,...,...,...,...,...,...,...
2005,85.824535,14.228229,0,0,10,2,98,1.0,0.01,1.0,0.03,0.01
2006,85.978676,14.208278,0,0,10,1,99,1.0,0.01,1.0,0.03,0.01
2007,85.978676,14.208278,0,0,10,2,99,1.0,0.01,1.0,0.03,0.01
2008,85.701749,14.190396,0,0,10,1,100,1.0,0.01,1.0,0.03,0.01
